In [ ]:
!pip install igraph
!pip install leidenalg

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
folder = '/content/drive/MyDrive/Institutions/'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
folder = '/content/drive/My Drive/Colaboração/Institutional collaboration/'

In [ ]:
from igraph import Graph
import igraph as ig
import pandas as pd
import leidenalg as la
import numpy as np
import pandas_gbq
import glob
from google.cloud import bigquery
import matplotlib.pyplot as plt

In [ ]:
# engineering = 22
# physics = 31
# medicine = 27
# computing = 17

area_codes = [22,31,27,17]
areas = ['Engineering','Physics','Medicine','Computing']
selected = 3
area = areas[selected]
area_code = area_codes[selected]

In [ ]:
client = bigquery.Client(project='insypo')
job = client.query('''
  DROP TABLE IF EXISTS projectdb_institutional_collaboration.works_fields;
  CREATE TABLE projectdb_institutional_collaboration.works_fields AS (
    SELECT DISTINCT a.work_id, c.publication_year
    FROM `insyspo.publicdb_openalex_2025_03_rm.works_topics` AS a
    JOIN `insyspo.publicdb_openalex_2025_03_rm.topics` AS b
    ON a.topic_id = b.id
    JOIN `insyspo.publicdb_openalex_2025_03_rm.works` AS c
    ON a.work_id = c.id
    WHERE b.field = %s AND publication_year >= 1990 AND c.type = 'article'
      AND a.score >= 0.9
  );
''' % (str(area_code)),project='insyspo')
print(job)

In [ ]:
folder

In [ ]:
%%bigquery --project insyspo

SELECT COUNT(*) AS works
FROM projectdb_institutional_collaboration.works_fields

In [ ]:
%%bigquery --project insyspo

CREATE OR REPLACE TABLE projectdb_institutional_collaboration.relevant_institutions AS (
  SELECT DISTINCT c.*
  FROM projectdb_institutional_collaboration.works_fields AS a
  JOIN `insyspo.publicdb_openalex_2025_03_rm.works_authorships` AS b
  ON a.work_id = b.work_id
  JOIN `insyspo.publicdb_openalex_2025_03_rm.institutions` AS c
  ON b.institution_id = c.id
  WHERE c.works_count > 10
)

In [ ]:
%%bigquery --project insyspo

CREATE OR REPLACE TABLE projectdb_institutional_collaboration.relevant_institutions_works AS (
  SELECT d.id, COUNT(DISTINCT(a.work_id)) AS n
  FROM projectdb_institutional_collaboration.works_fields AS a
  JOIN `insyspo.publicdb_openalex_2025_03_rm.works_authorships` AS b
  ON a.work_id = b.work_id
  JOIN `insyspo.publicdb_openalex_2025_03_rm.institutions` AS c
  ON b.institution_id = c.id
  JOIN projectdb_institutional_collaboration.relevant_institutions AS d
  ON c.id = d.id
  GROUP BY d.id
)

In [ ]:
%%bigquery --project insyspo

CREATE OR REPLACE TABLE projectdb_institutional_collaboration.relevant_institutions_filtered_largest AS (
  SELECT id
  FROM (
    SELECT *,
      NTILE(100) OVER (ORDER BY n DESC) AS ptile,
    FROM projectdb_institutional_collaboration.relevant_institutions_works
    ORDER BY n DESC
  )
  WHERE ptile <= 10
);

CREATE OR REPLACE TABLE projectdb_institutional_collaboration.relevant_institutions_filtered_smallest AS (
  SELECT id
  FROM (
    SELECT *,
      NTILE(100) OVER (ORDER BY n DESC) AS ptile,
    FROM projectdb_institutional_collaboration.relevant_institutions_works
    ORDER BY n DESC
  )
  WHERE ptile > 10 AND n>10
);

In [ ]:
%%bigquery df --project insyspo

SELECT b.country_code, COUNT(DISTINCT(a.id)) AS institutions
FROM projectdb_institutional_collaboration.relevant_institutions_filtered_largest AS a
JOIN publicdb_openalex_2025_03_rm.institutions AS b
ON a.id = b.id
WHERE b.country_code IS NOT NULL
GROUP BY b.country_code
ORDER BY institutions DESC
LIMIT 10

In [ ]:
df.to_csv(folder+area+'_country_institutions_largest.csv')

In [ ]:
%%bigquery df --project insyspo

SELECT b.country_code, COUNT(DISTINCT(a.id)) AS institutions
FROM projectdb_institutional_collaboration.relevant_institutions_filtered_smallest AS a
JOIN publicdb_openalex_2025_03_rm.institutions AS b
ON a.id = b.id
WHERE b.country_code IS NOT NULL
GROUP BY b.country_code
ORDER BY institutions DESC
LIMIT 10

In [ ]:
df.to_csv(folder+area+'_country_institutions_smallest.csv')

In [ ]:
%%bigquery df --project insyspo

SELECT b.country_code, COUNT(DISTINCT(a.id)) AS institutions
FROM projectdb_institutional_collaboration.relevant_institutions AS a
JOIN publicdb_openalex_2025_03_rm.institutions AS b
ON a.id = b.id
WHERE b.country_code IS NOT NULL
GROUP BY b.country_code
ORDER BY institutions DESC
LIMIT 10

In [ ]:
df.to_csv(folder+area+'_country_institutions.csv')

In [ ]:
%%bigquery df --project insyspo

SELECT COUNT(*)
FROM projectdb_institutional_collaboration.relevant_institutions_filtered_largest


In [ ]:
df.to_csv(folder+area+'_largest_institutions.csv')

In [ ]:
%%bigquery df --project insyspo

SELECT COUNT(*)
FROM projectdb_institutional_collaboration.relevant_institutions_filtered_smallest


In [ ]:
df.to_csv(folder+area+'_smallest_institutions.csv')

In [ ]:
%%bigquery --project insyspo

CREATE OR REPLACE TABLE projectdb_institutional_collaboration.works_authorships_large AS (
  SELECT b.*
  FROM projectdb_institutional_collaboration.relevant_institutions_filtered_largest AS a
  JOIN `insyspo.publicdb_openalex_2025_03_rm.works_authorships` AS b
  ON a.id = b.institution_id
);

CREATE OR REPLACE TABLE projectdb_institutional_collaboration.works_authorships_small AS (
  SELECT b.*
  FROM projectdb_institutional_collaboration.relevant_institutions_filtered_smallest AS a
  JOIN `insyspo.publicdb_openalex_2025_03_rm.works_authorships` AS b
  ON a.id = b.institution_id
);

CREATE OR REPLACE TABLE projectdb_institutional_collaboration.works_authorships AS (
  SELECT b.*
  FROM projectdb_institutional_collaboration.relevant_institutions AS a
  JOIN `insyspo.publicdb_openalex_2025_03_rm.works_authorships` AS b
  ON a.id = b.institution_id
);

In [ ]:
%%bigquery --project insyspo

DROP TABLE IF EXISTS projectdb_institutional_collaboration.insts_colabs_large;
CREATE TABLE projectdb_institutional_collaboration.insts_colabs_large AS (
  SELECT b.institution_id AS insta,
    c.institution_id AS instb, COUNT(DISTINCT(a.work_id)) AS works
  FROM `insyspo.projectdb_institutional_collaboration.works_fields` AS a
  JOIN `projectdb_institutional_collaboration.works_authorships_large` AS b
  ON a.work_id = b.work_id
  JOIN `projectdb_institutional_collaboration.works_authorships_large` AS c
  ON a.work_id = c.work_id
  WHERE b.institution_id > c.institution_id
  GROUP BY b.institution_id, c.institution_id
);

DROP TABLE IF EXISTS projectdb_institutional_collaboration.insts_colabs_small;
CREATE TABLE projectdb_institutional_collaboration.insts_colabs_small AS (
  SELECT b.institution_id AS insta,
    c.institution_id AS instb, COUNT(DISTINCT(a.work_id)) AS works
  FROM `insyspo.projectdb_institutional_collaboration.works_fields` AS a
  JOIN `projectdb_institutional_collaboration.works_authorships_small` AS b
  ON a.work_id = b.work_id
  JOIN `projectdb_institutional_collaboration.works_authorships_small` AS c
  ON a.work_id = c.work_id
  WHERE b.institution_id > c.institution_id
  GROUP BY b.institution_id, c.institution_id
);

DROP TABLE IF EXISTS projectdb_institutional_collaboration.insts_colabs;
CREATE TABLE projectdb_institutional_collaboration.insts_colabs AS (
  SELECT b.institution_id AS insta,
    c.institution_id AS instb, COUNT(DISTINCT(a.work_id)) AS works
  FROM `insyspo.projectdb_institutional_collaboration.works_fields` AS a
  JOIN `projectdb_institutional_collaboration.works_authorships` AS b
  ON a.work_id = b.work_id
  JOIN `projectdb_institutional_collaboration.works_authorships` AS c
  ON a.work_id = c.work_id
  WHERE b.institution_id > c.institution_id
  GROUP BY b.institution_id, c.institution_id
);

In [ ]:
%%bigquery --project insyspo

DROP TABLE IF EXISTS projectdb_institutional_collaboration.insts_colabs_year_large;
CREATE TABLE projectdb_institutional_collaboration.insts_colabs_year_large AS (
  SELECT b.institution_id AS inst1,
    c.institution_id AS inst2,
    a.publication_year
  FROM `insyspo.projectdb_institutional_collaboration.works_fields` AS a
  JOIN `projectdb_institutional_collaboration.works_authorships_large` AS b
  ON a.work_id = b.work_id
  JOIN `projectdb_institutional_collaboration.works_authorships_large` AS c
  ON a.work_id = c.work_id
  WHERE b.institution_id > c.institution_id
  GROUP BY b.institution_id, c.institution_id, a.publication_year
);

DROP TABLE IF EXISTS projectdb_institutional_collaboration.insts_colabs_year_small;
CREATE TABLE projectdb_institutional_collaboration.insts_colabs_year_small AS (
  SELECT b.institution_id AS inst1,
    c.institution_id AS inst2,
    a.publication_year
  FROM `insyspo.projectdb_institutional_collaboration.works_fields` AS a
  JOIN `projectdb_institutional_collaboration.works_authorships_small` AS b
  ON a.work_id = b.work_id
  JOIN `projectdb_institutional_collaboration.works_authorships_small` AS c
  ON a.work_id = c.work_id
  WHERE b.institution_id > c.institution_id
  GROUP BY b.institution_id, c.institution_id, a.publication_year
);

DROP TABLE IF EXISTS projectdb_institutional_collaboration.insts_colabs_year;
CREATE TABLE projectdb_institutional_collaboration.insts_colabs_year AS (
  SELECT b.institution_id AS inst1,
    c.institution_id AS inst2,
    a.publication_year
  FROM `insyspo.projectdb_institutional_collaboration.works_fields` AS a
  JOIN `projectdb_institutional_collaboration.works_authorships` AS b
  ON a.work_id = b.work_id
  JOIN `projectdb_institutional_collaboration.works_authorships` AS c
  ON a.work_id = c.work_id
  WHERE b.institution_id > c.institution_id
  GROUP BY b.institution_id, c.institution_id, a.publication_year
);

In [ ]:
%%bigquery df --project insyspo

SELECT decade, COUNT(*) AS works
FROM (
  SELECT *,
    IF(publication_year>=1990 AND publication_year<=1999,1990,
      IF(publication_year>=2000 AND publication_year<=2009,2000,
        IF(publication_year>=2010 AND publication_year<=2019,2010,NULL))) AS decade
  FROM projectdb_institutional_collaboration.insts_colabs_year_small
  WHERE publication_year <=2019
)
GROUP BY decade


In [ ]:
df.to_csv(folder+area+'collabs_small.csv')

In [ ]:
%%bigquery df --project insyspo

SELECT decade, COUNT(*) AS works
FROM (
  SELECT *,
    IF(publication_year>=1990 AND publication_year<=1999,1990,
      IF(publication_year>=2000 AND publication_year<=2009,2000,
        IF(publication_year>=2010 AND publication_year<=2019,2010,NULL))) AS decade
  FROM projectdb_institutional_collaboration.insts_colabs_year
  WHERE publication_year <=2019
)
GROUP BY decade


In [ ]:
df.to_csv(folder+area+'collabs.csv')

In [ ]:
%%bigquery df --project insyspo

SELECT decade, COUNT(*) AS works
FROM (
  SELECT *,
    IF(publication_year>=1990 AND publication_year<=1999,1990,
      IF(publication_year>=2000 AND publication_year<=2009,2000,
        IF(publication_year>=2010 AND publication_year<=2019,2010,NULL))) AS decade
  FROM projectdb_institutional_collaboration.insts_colabs_year_large
  WHERE publication_year <=2019
)
GROUP BY decade


In [ ]:
df.to_csv(folder+area+'collabs_large.csv')

# Networks' analysis

In [ ]:
!pip install igraph
!pip install leidenalg

In [ ]:
import pandas as pd
import pandas_gbq as pgbq
import igraph as ig
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
intervals = [1990, 2000, 2010]
di = 10
G = [[],[],[]]

connections =  pd.DataFrame({'inst1': [],
                                'inst2': [],
                                'weight': [],
                                'interval': []})

connections['inst1'] = connections['inst1'].astype(int)
connections['inst2'] = connections['inst2'].astype(int)
connections['interval'] = connections['interval'].astype(int)
connections['weight'] = connections['weight'].astype(float)

for ci, i in enumerate(intervals):
  query = '''
  SELECT inst1, inst2,
    COUNT(*)/(
        SELECT COUNT(*)
        FROM insyspo.projectdb_institutional_collaboration.insts_colabs_year
        WHERE publication_year >= {0} AND publication_year < {1}
    ) AS weight
  FROM insyspo.projectdb_institutional_collaboration.insts_colabs_year
  WHERE publication_year >= {0} AND publication_year < {1} AND inst1 > inst2
  GROUP BY inst1, inst2
  '''.format(i,i+di)
  df = pd.read_gbq(query,project_id = 'insyspo')

  tuples = [tuple(x) for x in df.values]
  G[ci] = ig.Graph.TupleList(tuples, directed = False)

In [ ]:
%%bigquery df --project insyspo

SELECT *
FROM projectdb_institutional_collaboration.insts_colabs_year

In [ ]:
g1 = G[0]
g2 = G[1]
g3 = G[2]

degreeList1 = g1.degree()
degreeList2 = g2.degree()
degreeList3 = g3.degree()

markers = ['x', '.', '^']
labels = ['1990', '2000', '2010']

for degreeList, marker, label in zip([degreeList1, degreeList2, degreeList3], markers, labels):
    densityList = []
    degreeL = []

    for degree in set(degreeList):
        contagem = degreeList.count(degree)
        density = contagem / len(degreeList)
        degreeL.append(degree)
        densityList.append(density)
    print("Mean degree at ", label, " :", np.mean(degreeList))
    plt.loglog(degreeL, densityList, marker=marker, linestyle='',\
               label=label)


plt.xlabel('k')
plt.ylabel('p(k)')
plt.legend()
plt.title('Degree distribution - '+area)
plt.grid(True)
plt.savefig('degree_distribution_'+area+'.png')

# Hierarchy analysis

##Large institutions

In [ ]:
intervals = [1990, 2000, 2010]
di = 10
Gl = [[],[],[]]

connections =  pd.DataFrame({'inst1': [],
                                'inst2': [],
                                'weight': [],
                                'interval': []})

connections['inst1'] = connections['inst1'].astype(int)
connections['inst2'] = connections['inst2'].astype(int)
connections['interval'] = connections['interval'].astype(int)
connections['weight'] = connections['weight'].astype(float)

for ci, i in enumerate(intervals):
  query = '''
  SELECT inst1, inst2,
    COUNT(*)/(
        SELECT COUNT(*)
        FROM insyspo.projectdb_institutional_collaboration.insts_colabs_year_large
        WHERE publication_year >= {0} AND publication_year < {1}
    ) AS weight
  FROM insyspo.projectdb_institutional_collaboration.insts_colabs_year_large
  WHERE publication_year >= {0} AND publication_year < {1} AND inst1 > inst2
  GROUP BY inst1, inst2
  '''.format(i,i+di)
  df = pd.read_gbq(query,project_id = 'insyspo')

  tuples = [tuple(x) for x in df.values]
  Gl[ci] = ig.Graph.TupleList(tuples, directed = False)

In [ ]:
g1 = Gl[0]
g2 = Gl[1]
g3 = Gl[2]

degreeList1 = g1.degree()
degreeList2 = g2.degree()
degreeList3 = g3.degree()

markers = ['x', '.', '^']
labels = ['1990', '2000', '2010']

for degreeList, marker, label in zip([degreeList1, degreeList2, degreeList3], markers, labels):
    densityList = []
    degreeL = []

    for degree in set(degreeList):
        contagem = degreeList.count(degree)
        density = contagem / len(degreeList)
        degreeL.append(degree)
        densityList.append(density)
    print("Mean degree at ", label," :", np.mean(degreeList))
    plt.loglog(degreeL, densityList, marker=marker, linestyle='',\
               label=label)

plt.xlabel('k')
plt.ylabel('p(k)')
plt.legend()
plt.title('Degree distribution - '+area+' - large institutions')
plt.grid(True)
plt.savefig('degree_distribution_'+area+'_large.png')

## Small institutions

In [ ]:
intervals = [1990, 2000, 2010]
di = 10
Gs = [[],[],[]]

connections =  pd.DataFrame({'inst1': [],
                                'inst2': [],
                                'weight': [],
                                'interval': []})

connections['inst1'] = connections['inst1'].astype(int)
connections['inst2'] = connections['inst2'].astype(int)
connections['interval'] = connections['interval'].astype(int)
connections['weight'] = connections['weight'].astype(float)

for ci, i in enumerate(intervals):
  query = '''
  SELECT inst1, inst2,
    COUNT(*)/(
        SELECT COUNT(*)
        FROM insyspo.projectdb_institutional_collaboration.insts_colabs_year_small
        WHERE publication_year >= {0} AND publication_year < {1}
    ) AS weight
  FROM insyspo.projectdb_institutional_collaboration.insts_colabs_year_small
  WHERE publication_year >= {0} AND publication_year < {1} AND inst1 > inst2
  GROUP BY inst1, inst2
  '''.format(i,i+di)
  df = pd.read_gbq(query,project_id = 'insyspo')

  tuples = [tuple(x) for x in df.values]
  Gs[ci] = ig.Graph.TupleList(tuples, directed = False)

In [ ]:
g1 = Gs[0]
g2 = Gs[1]
g3 = Gs[2]

degreeList1 = g1.degree()
degreeList2 = g2.degree()
degreeList3 = g3.degree()

markers = ['x', '.', '^']
labels = ['1990', '2000', '2010']

for degreeList, marker, label in zip([degreeList1, degreeList2, degreeList3], markers, labels):
    densityList = []
    degreeL = []

    for degree in set(degreeList):
        contagem = degreeList.count(degree)
        density = contagem / len(degreeList)
        degreeL.append(degree)
        densityList.append(density)
    print("Mean degree at ", label," :", np.mean(degreeList))
    plt.loglog(degreeL, densityList, marker=marker, linestyle='',\
               label=label)

plt.xlabel('k')
plt.ylabel('p(k)')
plt.legend()
plt.title('Degree distribution - '+area+' - small institutions')
plt.grid(True)
plt.savefig('degree_distribution_'+area+'_small.png')